## Prepare CMIP6 Data for ML Models

In [90]:
# import packages

import numpy as np
import xarray as xr
import pandas as pd

# plotting
try:
    import matplotlib.pyplot as plt
    import cartopy.crs as ccrs
except:
    print('not installed')

In [115]:
# define path to CMIP6 resolution 5°
pathway = 'ssp585' # CHANGE HERE
model = 'mri-esm2' # CHANGE HERE

datadir = '/home/steidani/hackathon/2021_ai_climate/data/cmip6/' + model + '/'

In [116]:
t2m = xr.open_mfdataset(f'{datadir}' + model + '-0_r1i1p1f1_w5e5_' + pathway + '_tas_global_daily_*.nc', combine='by_coords').tas
precip = xr.open_mfdataset(f'{datadir}' + model + '-0_r1i1p1f1_w5e5_' + pathway + '_pr_global_daily_*.nc', combine='by_coords').pr

In [117]:
# units:

# t2m in K
# precip in kg m-2 s-1 (flux)

# ERA5 has precip unit m (sum over one day): convert precip unit
#  1 kg/m2/s = 86400 mm/day = 86.4 m / day
precip = precip * 86.4

In [118]:
# make monthly means for each year
t2m_mean = t2m.resample(time='1M', keep_attrs=True).mean(keep_attrs=True).dropna('time', how='all')
t2m_std = t2m.resample(time='1M', keep_attrs=True).std(keep_attrs=True).dropna('time', how='all')
precip_mean = precip.resample(time='1M', keep_attrs=True).mean(keep_attrs=True).dropna('time', how='all')
precip_std = precip.resample(time='1M', keep_attrs=True).std(keep_attrs=True).dropna('time', how='all')

In [119]:
# function to get the lat lon boundary of the MODIS tiles (h,v)

def get_lat_lon(h,v):  
    def _get_sinu_grid_df():
        from pandas import read_csv
        f = 'sn_bound_10deg.txt'
        td = read_csv(f, skiprows=5, delim_whitespace=True)
        td = td.assign(ihiv='h' + td.ih.astype(str).str.zfill(2) +
                       'v' + td.iv.astype(str).str.zfill(2))
        return td

    td = _get_sinu_grid_df()
    o = td.loc[(td.ih == int(h)) & (td.iv == int(v))]
    latmin = o.lat_min.iloc[0]
    lonmin = o.lon_min.iloc[0]
    latmax = o.lat_max.iloc[0]
    lonmax = o.lon_max.iloc[0]
    
    return lonmin, latmin, lonmax, latmax    

In [120]:
# get unique tiles (combination of h and v)
fire_counts=pd.read_csv('/home/steidani/hackathon/2021_ai_climate/data/MCD64A1/fire_counts_from_mcd64a1_meta.csv')
hs_vs = fire_counts[["h",'v']].drop_duplicates().sort_values(['h', 'v'])

vs = []
hs = []
mean_t2m = []
std_t2m = []
mean_precip = []
std_precip = []
years = []
months = [] 
#years = t2m_mean.time['time.year'].values
#months = t2m_mean.time['time.month'].values
# loop through each unique tile and get temporal evolution of precip and temp:
for index, row in hs_vs.iterrows():
    h = row['h']
    v = row['v']
    
    # tile boundary in lat lon
    lonmin, latmin, lonmax, latmax = get_lat_lon(h,v)
    
    # mean and std value
    _mean_t2m = t2m_mean.sel(lat=slice(latmin,latmax), lon=slice(lonmin,lonmax)).mean(("lon", "lat"),skipna=True).values
    _std_t2m = t2m_std.sel(lat=slice(latmin,latmax), lon=slice(lonmin,lonmax)).mean(("lon", "lat")).values
    
    _mean_precip = precip_mean.sel(lat=slice(latmin,latmax), lon=slice(lonmin,lonmax)).mean(("lon", "lat"),skipna=True).values
    _std_precip = precip_std.sel(lat=slice(latmin,latmax), lon=slice(lonmin,lonmax)).mean(("lon", "lat")).values
                      
    # append to list
    vs.extend(np.repeat(v, len(_mean_t2m)))
    hs.extend(np.repeat(h, len(_mean_t2m)))
    years.extend(t2m_mean.time['time.year'].values)
    months.extend(t2m_mean.time['time.month'].values)
    mean_t2m.extend(_mean_t2m)
    std_t2m.extend(_std_t2m)
    mean_precip.extend(_mean_precip)
    std_precip.extend(_std_precip)

/home/steidani/miniconda3/envs/ai_climate_2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/home/steidani/miniconda3/envs/ai_climate_2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/home/steidani/miniconda3/envs/ai_climate_2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/home/steidani/miniconda3/envs/ai_climate_2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/home/steidani/miniconda3/envs/ai_climate_2021/lib/python3.8/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/home/steidani/miniconda3/envs/ai_climat

In [121]:
# quick check
print(len(months))
len(months) == len(years) == len(vs) == len(mean_t2m) == len(mean_precip)

64320


True

In [122]:
# data to dataframe
input_csv = pd.DataFrame(sorted(list(zip(vs,hs,years,months,mean_t2m,std_t2m, mean_precip, std_precip)), key=lambda x: (x[0], x[1])) , columns=['v','h','year','month','t2m_mean','t2m_std', 'precip_mean', 'precip_std'])
input_csv = input_csv.dropna() # unfortunately, some tiles are to small (at the edge)?
input_csv

,v,h,year,month,t2m_mean,t2m_std,precip_mean,precip_std
0,2,9,2051,1,268.706085,4.029854,0.002143,0.003096
1,2,9,2051,2,265.026031,4.076586,0.001627,0.003069
2,2,9,2051,3,264.724915,3.512778,0.001768,0.002871
3,2,9,2051,4,268.988159,3.209100,0.002983,0.004535
4,2,9,2051,5,274.280334,3.080663,0.002706,0.004281
...,...,...,...,...,...,...,...,...
64315,14,14,2100,8,277.958008,1.347812,0.003071,0.003692
64316,14,14,2100,9,279.675842,0.792601,0.003053,0.004892
64317,14,14,2100,10,279.604675,1.006945,0.003321,0.004261
64318,14,14,2100,11,280.725861,0.921595,0.003683,0.006172


In [123]:
# save as csv
input_csv.round(4).to_csv('/home/steidani/hackathon/2021_ai_climate/data/cmip6_' + pathway + '_' + model + '.csv',index=False)